<center><h1>Обучение с подкреплением<h1/><center/>

<img src="https://sun9-8.userapi.com/impg/zOa9OmJAuFXbDMEvcWVmwp45e2qvJ5MxojsyZw/Hta-Zpo2KSE.jpg?size=1080x1005&quality=96&sign=3320b562fcdb238d4c5a621b55fa8c9c&type=album" height=720 width=670/>

## Инициализация игры

In [1]:
from sklearn.preprocessing import KBinsDiscretizer
from collections import defaultdict
import gym
import statistics
import random
import numpy as np
import math

In [2]:
env = gym.make('CartPole-v1')

Данная игра представляет из себя каретку, на которой расположена палка в состоянии неравновесия. Наша цель – не позволить палке упасть. Мы можем достичь этого, двигая каретку влево либо вправо. Посмотрим, как выглядит эта игра.

In [3]:
for _ in range(3):
    observation = env.reset()
    for t in range(200):
        env.render()
        a = env.action_space.sample()
        env.step(a)
env.close()

c:\users\alekc\appdata\local\programs\python\python38\lib\site-packages\gym\envs\classic_control\cartpole.py:150: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


Попробуем вручную подобрать тактику. Пусть каждый раз, когда палка отклоняется влево, каретка также начинается двигаться влево и наоборот.

In [4]:
for _ in range(5):
    observation = env.reset()
    for t in range(200):
        env.render()
        action = 0 if observation[2] < 0 else 1
        observation, reward, done, info = env.step(action)
env.close()

## Q-Learning

Для того чтобы обучить нашего агента, будем использовать Q-Learning.

In [5]:
class QLearningAgent():

    def __init__(self, discount, 
                 get_legal_actions):
        self.get_legal_actions = get_legal_actions
        self._q_values = \
            defaultdict(lambda: defaultdict(lambda: 0))  
        self.discount = discount

    def get_q_value(self, state, action):
        return self._q_values[state][action]

    def set_q_value(self, state, action, value):
        self._q_values[state][action] = value
        
    def _calculate_alpha(self, n, min_rate=0.01 ) -> float  :
        """Адаптивно считает гиперпараметр альфа (коэффициент обучаемости)"""
        
        return max(min_rate, min(1.0, 1.0 - math.log10((n + 1) / 25)))
    
    def _calculate_epsilon(self, n, min_rate=0.1 ) -> float  :
        """Адаптивно считает гиперпараметр эпсилон (коэффициент случайности)"""
        return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))
        
    def get_value(self, state):
        """
          Возвращает значение функции полезности, 
          рассчитанной по Q[state, action]
        """
        possible_actions = self.get_legal_actions(state)
        value = max([self.get_q_value(state, action) for action in possible_actions])
        return value

    def get_policy(self, state):
        """
          Выбирает лучшее действие, согласно стратегии.
        """
        possible_actions = self.get_legal_actions(state)
        
        if all( self.get_q_value(state, action) ==  self.get_q_value(state, possible_actions[0]) for action in possible_actions):
            return random.choice(possible_actions)

        best_action = None
        for action in possible_actions:
            if best_action is None:
                best_action = action
            elif self.get_q_value(state, action) > self.get_q_value(state, best_action):
                best_action = action
        return best_action
        
    
    def get_action(self, n, state):
        """
          Выбирает действие, предпринимаемое в данном 
          состоянии, включая исследование (eps greedy)
          С вероятностью self.epsilon берем случайное 
          действие, иначе действие согласно стратегии 
          (self.get_policy)
        """
        possible_actions = self.get_legal_actions(state)

        if np.random.random() < self._calculate_epsilon(n):
            action = np.random.choice(possible_actions, 1)[0]
        else:
            action = self.get_policy(state)

        return action

    def update(self, n, state, action, next_state, reward):
        """
          Функция Q-обновления 
        """
        alpha = self._calculate_alpha(n)
        
        learnt_value = reward + self.discount * self.get_value(next_state)
        old_value = self.get_q_value(state, action)
        reference_qvalue = (1 - alpha) * old_value + alpha * learnt_value
        
        self.set_q_value(state, action, reference_qvalue)

## Предобработка данных

Обучение с дообучением может быть использовано лишь на конечном множестве состояний. Однако положение палки – величина дискретная. Поэтому нам необходимо разбить наше дискретной множество признаков на конечное (и относительно небольшое) количество состояний-комбинаций.

In [6]:
n_bins = (6 , 12)
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
est.fit([lower_bounds, upper_bounds ])

def discreditize(obs):
    return tuple(map(int,est.transform([[obs[2], obs[3]]])[0]))

## Непосредственно обучение

In [7]:
def render_in_range(env: gym.wrappers.time_limit.TimeLimit, pos: int, range_: tuple):
    if range_[0] <= pos < range_[1]:
        env.render()
    elif pos == range_[1]:
        env.close()

In [8]:
agent = QLearningAgent(discount=1,
                       get_legal_actions=lambda s: range(
                           env.action_space.n))

In [ ]:
for iteration in range(1000):       
    total_reward, done = 0, False
    observation = env.reset()
    
    while not done:
        angle = discreditize(observation)

        a = agent.get_action(iteration, angle)
        next_observation, reward, done, _ = env.step(a)

        next_angle = discreditize(next_observation)
        
        agent.update(iteration, angle, a, next_angle, reward) 
        
#         reward *= 1 - abs(observation[0]) / 4.8
        
        render_in_range(env, iteration, (100, 105))
        
        render_in_range(env, iteration, (200, 204))
        
        render_in_range(env, iteration, (500, 503))

        observation = next_observation
        
        total_reward += reward

            
    if iteration % 50 == 0 and iteration != 0:
        print(total_reward)


11.0
215.0
58.0
500.0
500.0
456.0
416.0
500.0
